<a href="https://colab.research.google.com/github/eminshall/FreeCodeCamp-Projects-TensorFlow/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#I relied heavily on YouTube videos and other notebooks I found in Github to code this project.
#I did my best to make the code my own, apologies in advance if it wasn't enough.
#I'd include contributors, but there were dozens.

# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
#Import more libraries
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import sequence

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
#Make Data Frames
df_train = pd.read_csv(test_file_path, sep="\t", header=None, names=["type", "msg"])

df_test = pd.read_csv(test_file_path, sep="\t", header=None, names=["type", "msg"])

In [ ]:
df_train.isna().sum()

In [ ]:
df_test.isna().sum()

In [ ]:
df_train.head(3)

In [ ]:
df_test.head(3)

In [ ]:
#Prepare Labels
df_train["type"] = pd.factorize(df_train["type"])[0]
df_test["type"] = pd.factorize(df_test["type"])[0]

In [ ]:
df_train.head()

In [ ]:
train_labels =  df_train["type"].values
train_ds = tf.data.Dataset.from_tensor_slices(
    (df_train["msg"].values, train_labels)
)

In [ ]:
test_labels = df_test["type"].values
test_ds = tf.data.Dataset.from_tensor_slices(
    (df_test["msg"].values, test_labels)
)
test_ds.element_spec

In [ ]:
#Set Batching
buffer = 100
batch_size = 32
train_ds = train_ds.shuffle(buffer).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Text vectorization
textvector = TextVectorization(
    output_mode='int',
    max_tokens=1000,
    output_sequence_length=1000,
)

textvector.adapt(train_ds.map(lambda text, label: text))

In [ ]:
#Prepare data
# Text vectorization: Vocabulary learnt by our vectorizer
vocab = np.array(textvector.get_vocabulary())
vocab[:20]

In [ ]:
# Create the model
model = tf.keras.Sequential([
    textvector,
    tf.keras.layers.Embedding(
        len(textvector.get_vocabulary()),
        64,
        mask_zero=True,
    ),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1)
])


model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=['accuracy'],
)

In [ ]:
# Train the model against data sets
history = model.fit(
    train_ds,
    validation_data=test_ds,
    validation_steps=30,
    epochs=10,
)

In [ ]:
test_loss, test_acc = model.evaluate(test_ds)

In [ ]:
# Plot the accuracy and loss metrics
# Create a helper function to plot
def plot_graphs(h, metric):
    plt.plot(h.history[metric])
    plt.plot(h.history['val_'+metric])
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])

In [ ]:
# Plot the accuracy and loss metrics
# Plot the graphs
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')
plt.ylim(0, None) 

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    ps = model.predict([pred_text])
    print(ps)
    p = ps[0][0]
    return [p, "ham" if p <0.5 else "spam"]

    pred_text = "how are you doing today?"

    prediction = predict_message(pred_text)
    print(prediction)

    return (prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
